In [ ]:
pip install plotly_express

In [ ]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import plotly_express as px

import math
import seaborn as sns
import time
start_run = time.time()

#Analysis, patterns and Correlations
from sklearn.decomposition import PCA

# preprocessing
import sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf


#Neural network
import keras
from keras import optimizers
from keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Flatten, BatchNormalization, Activation
from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Flatten, BatchNormalization, Activation
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.callbacks import EarlyStopping



#Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report, recall_score, precision_score, f1_score, roc_curve, auc

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
q_data = pd.read_csv("Quitline___Services_Available___Counseling_-_2010_To_Present_20240208.csv")

In [ ]:
q_data.head()

In [ ]:
q_data.isnull().sum()

In [ ]:
q_data.shape

(5230, 32)

In [ ]:
q_data['Number_of_Sessions_Offered'].unique().shape

In [ ]:
import pandas as pd

# Calculate the mean value of the column
mean_sessions = q_data['Number_of_Sessions_Offered'].mean()

# Replace missing values with the mean
q_data['Number_of_Sessions_Offered'].fillna(mean_sessions, inplace=True)

# Analyzing Number_of_Sessions_Offered column
print('value counts in Number_of_Sessions_Offered column in percentage')
print('-'*65)
display(q_data['Number_of_Sessions_Offered'].value_counts(normalize=True)*100)

In [ ]:
q_data2 = q_data.drop(['LocationAbbr', 'TopicType', 'TopicDesc', 'MeasureDesc',
                      'Sub-Measure', 'Geographic_Requirements_Text', 'Age_Requirements_Text',
                      'RQUIT_Requirements_Text', 'Insurance_Requirements_Text', 'Pregnancy_Requirements_Text',
                      'Other_Requirements_Text', 'Comments',
                      'TopicTypeId', 'TopicId', 'MeasureID', 'SOURCE', 'SubMeasureID', 'DisplayOrder'], axis=1)
q_data2.head()

In [ ]:
q_data2.shape

In [ ]:
q_data2.info()

In [ ]:
q_tab = q_data2.copy()

In [ ]:
# Splitting the 'Geolocation' column into separate 'latitude' and 'longitude' columns
q_tab[['latitude', 'longitude']] = q_tab['GeoLocation'].str.strip('()').str.split(',', expand=True)

# Convert the latitude and longitude columns to numeric type
q_tab['latitude'] = pd.to_numeric(q_tab['latitude'])
q_tab['longitude'] = pd.to_numeric(q_tab['longitude'])
q_tab.head()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Export the modified DataFrame to a CSV file in Google Drive for Tableau Geolocation Analysis
#q_tab.to_csv('/content/drive/My Drive/geolocation_data.csv', index=False)

In [ ]:
def counts_and_percentage(dataset, column):

    count = dataset[column].value_counts()
    percentage = dataset[column].value_counts(normalize=True) * 100

    return pd.concat([count,percentage.round(1)], axis=1, keys=['Count', 'Percentage'])

In [ ]:
counts_and_percentage(q_data2, 'Number_of_Sessions_Offered')

In [ ]:
counts_and_percentage(q_data2, 'Year')

In [ ]:
counts_and_percentage(q_data2, 'Date')

In [ ]:
counts_and_percentage(q_data2, 'Date_Ref')

Date and Date_Ref are same
1-4 represent Quarters and 6 and 12 represent Halves of the year

In [ ]:
#Year vs count()
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = q_data2.sort_values('Year', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Date_Ref')):
  _plot_series(series, series_name, i)
  fig.legend(title='Date_Ref', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

In [ ]:
counts_and_percentage(q_data2, 'LocationDesc')

In [ ]:
q_data2['LocationDesc'].unique().shape

In [ ]:
counts_and_percentage(q_data2, 'Variable')

In [ ]:
q_data2['Variable'].unique().shape

In [ ]:
counts_and_percentage(q_data2, 'Geographic_Requirements')

In [ ]:
#Geographic_Requirements
q_data2.groupby('Geographic_Requirements').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
counts_and_percentage(q_data2, 'Age_Requirements')

In [ ]:
#Age_Requirements
q_data2.groupby('Age_Requirements').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#Year vs count()

def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = q_data2.sort_values('Year', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Age_Requirements')):
  _plot_series(series, series_name, i)
  fig.legend(title='Age_Requirements', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

In [ ]:
age_data = q_data2[q_data2['Age_Requirements'] == 'No']
age_data

In [ ]:
counts_and_percentage(q_data2, 'Insurance_Requirements')

In [ ]:
counts_and_percentage(q_data2, 'Pregnancy_Requirements')

In [ ]:
sns.countplot(x='Age_Requirements', hue='Pregnancy_Requirements', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
# @title Year vs count()


def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = q_data2.sort_values('Year', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Geographic_Requirements')):
  _plot_series(series, series_name, i)
  fig.legend(title='Geographic_Requirements', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

In [ ]:
counts_and_percentage(q_data2, 'Other_Requirements')

In [ ]:
counts_and_percentage(q_data2, 'Unlimited_Sessions_Offered')

In [ ]:
counts_and_percentage(q_data2, 'GeoLocation')

In [ ]:
q_data2['GeoLocation'].unique().shape

There are 53 US states and territories covered by this dataset

Same with states and territories

In [ ]:
q_data2.groupby('Date_Ref').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#Date
q_data2['Date'].plot(kind='hist', bins=20, title='Date')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#RQUIT_Requirements
q_data2.groupby('RQUIT_Requirements').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#QUIT_Requirements vs Insurance_Requirements
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Insurance_Requirements'].value_counts()
    for x_label, grp in q_data2.groupby('RQUIT_Requirements')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('RQUIT_Requirements')
_ = plt.ylabel('Insurance_Requirements')

In [ ]:
#Year vs count()

def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = q_data2.sort_values('Year', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('RQUIT_Requirements')):
  _plot_series(series, series_name, i)
  fig.legend(title='RQUIT_Requirements', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

In [ ]:
#Year
q_data2['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
counts_and_percentage(q_data2, 'RQUIT_Requirements')

In [ ]:
q_data2.shape

In [ ]:
q_data2.isnull().sum()

In [ ]:
def counts_and_percentage(dataset, column):

    count = dataset[column].value_counts()
    percentage = dataset[column].value_counts(normalize=True) * 100

    return pd.concat([count,percentage.round(1)], axis=1, keys=['Count', 'Percentage'])

In [ ]:
counts_and_percentage(q_data2, 'Year')

In [ ]:
Year_info = px.histogram(q_data2, x="Year", marginal="violin", template='ggplot2')
Year_info.update_layout(title='Year', title_x=0.5)

Year_info.show()
#Interactive diagram. Point on objects

In [ ]:
counts_and_percentage(q_data2, 'LocationDesc')

In [ ]:
counts_and_percentage(q_data2, 'Variable')

In [ ]:
sns.countplot(x='RQUIT_Requirements', hue='Pregnancy_Requirements', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
sns.countplot(x='RQUIT_Requirements', hue='Age_Requirements', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
sns.countplot(x='RQUIT_Requirements', hue='Insurance_Requirements', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
sns.countplot(x='RQUIT_Requirements', hue='Other_Requirements', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
sns.countplot(x='RQUIT_Requirements', hue='Unlimited_Sessions_Offered', palette= 'Set2', data=q_data2.sort_values(by=['RQUIT_Requirements']))
plt.show()

In [ ]:
pd.crosstab(q_data2['LocationDesc'], [q_data2['RQUIT_Requirements'], q_data2['Year']])

In [ ]:
#VISUALIZE THE FEATURES THROUGH PAIRPLOT
# Create a pair plot
sns.pairplot(q_data2, hue='RQUIT_Requirements')

# Display the plot
plt.show()

In [ ]:
q_data3 = q_data2.drop(['Date', 'Date_Ref', 'LocationDesc','Geographic_Requirements'], axis=1)
q_data3.head()

In [ ]:
q_data3.info()

In [ ]:
q_data3['Variable'].unique().shape

(90,)

In [ ]:
q_data3['Age_Requirements'].unique().shape

(2,)

In [ ]:
q_data3['Insurance_Requirements'].unique().shape

(2,)

In [ ]:
q_data3['Pregnancy_Requirements'].unique().shape

(2,)

In [ ]:
q_data3['Other_Requirements'].unique().shape

(2,)

In [ ]:
q_data3['Unlimited_Sessions_Offered'].unique().shape

(2,)

In [ ]:
q_data3['GeoLocation'].unique().shape

(53,)

In [ ]:
q_data3['Age_Requirements'].unique().shape

(2,)

In [ ]:
q_data3['Age_Requirements'].unique().shape

(2,)

In [ ]:
# Create a copy of the DataFrame to preserve q_data3 storage
q_data_encoded = q_data3.copy()

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate over each column
for column in q_data_encoded.columns:
    # Check if the column dtype is object (string)
    if q_data_encoded[column].dtype == 'object':
        # Use LabelEncoder to convert the string values to numerical values
        q_data_encoded[column] = label_encoder.fit_transform(q_data_encoded[column])

# Move 'RQUIT_Requirements' column to the last position
rquit_column = q_data_encoded.pop('RQUIT_Requirements')
q_data_encoded.insert(len(q_data_encoded.columns), 'RQUIT_Requirements', rquit_column)

In [ ]:
q_data_encoded.head()

In [ ]:
#VISUALIZE THE FEATURES THROUGH PAIRPLOT
# Create a pair plot
sns.pairplot(q_data_encoded, hue='RQUIT_Requirements')

# Display the plot
plt.show()

In [ ]:
q_data_encoded_no_year = q_data_encoded.drop(['Year'], axis=1)
q_data_encoded_no_year.head()

In [ ]:
import pandas as pd

# Calculate Spearman correlation for each variable with the target variable
correlations = q_data_encoded_no_year.corrwith(q_data_encoded_no_year['RQUIT_Requirements'], method='spearman')

# Create a DataFrame to store the correlations
correlation_df = pd.DataFrame(correlations, columns=['Spearman Correlation'])

# Sort the correlations by their absolute values in descending order
correlation_df['Absolute Correlation'] = correlation_df['Spearman Correlation'].abs()
correlation_df = correlation_df.sort_values(by='Absolute Correlation', ascending=False)

# Print the sorted correlations
print(correlation_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate Spearman correlation for all pairs of variables
correlation_matrix = q_data_encoded_no_year.corr(method='spearman')

# Plotting the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)
plt.title('Spearman Correlation Heatmap')
#plt.xlabel('Variables')
#plt.ylabel('Variables')
plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
Xsfs = q_data_encoded_no_year.iloc[:,0:8]
ysfs = q_data_encoded_no_year.iloc[:,-1]

In [ ]:
Xsfs.head()

In [ ]:
Xsfs.shape

(5230, 8)

In [ ]:
# Normalization for PCA analysis
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# Fitting the input data
q_scale = scale.fit_transform(Xsfs)

In [ ]:
# Instantiate PCA object
pca = PCA()

# Fit and transform the scaled data
pca_data = pca.fit_transform(q_scale)

# Plot explained variance ratio
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
plt.title('Explained Variance Ratio')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

# Assuming X is your data matrix
# Initialize PCA
pca = PCA()

# Fit PCA to your data
pca.fit(q_scale)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Plotting the cumulative explained variance
plt.plot(np.arange(1, len(cumulative_variance) + 1), cumulative_variance)
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.show()

In [ ]:
y = q_data_encoded.iloc[:,-1]

In [ ]:
# Convert 'Year' column to datetime format
q_data_encoded['Year'] = pd.to_datetime(q_data_encoded['Year'], format='%Y')

# Split dataset into training and testing sets
train_data = q_data_encoded[q_data_encoded['Year'].dt.year.isin(range(2010, 2018))]
test_data = q_data_encoded[q_data_encoded['Year'].dt.year.isin(range(2018, 2021))]

# Reset index
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data.shape

(4070, 10)

In [ ]:
train_data = train_data.drop(['Year'], axis=1)
train_data.shape

(4070, 9)

In [ ]:
test_data.head()

In [ ]:
test_data.shape

(1160, 10)

In [ ]:
test_data = test_data.drop(['Year'], axis=1)
test_data.shape

(1160, 9)

In [ ]:
test_data.head()

In [ ]:
# Encode categorical variables
label_encoder = LabelEncoder()
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        train_data[column] = label_encoder.fit_transform(train_data[column])

for column in test_data.columns:
    if test_data[column].dtype == 'object':
        test_data[column] = label_encoder.transform(test_data[column])

In [ ]:
# Split features and target variable for training and testing sets
X_train = train_data.drop(columns=['RQUIT_Requirements'])
y_train = train_data['RQUIT_Requirements']
X_test = test_data.drop(columns=['RQUIT_Requirements'])
y_test = test_data['RQUIT_Requirements']

# Scale features (excluding datetime columns)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(exclude=['datetime']))
X_test_scaled = scaler.transform(X_test.select_dtypes(exclude=['datetime']))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam

# Define the CNN model
cnn_model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    BatchNormalization(),  # Add batch normalization
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),  # Add batch normalization
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),  # Add batch normalization
    MaxPooling1D(pool_size= 1),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),  # Add dropout layer for regularization
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification
])


# Compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.summary()

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
history_cnn = cnn_model.fit(X_train_scaled, y_train, epochs=300, batch_size=32,
                            validation_split=0.2, verbose=0, callbacks=[early_stopping],
                            shuffle=True, use_multiprocessing = True)

# Evaluate the model on the test set
y_pred = (cnn_model.predict(X_test_scaled) > 0.5).astype("int32")

# Metrics
accuracy_cnn = round(accuracy_score(y_test, y_pred),4)
recall_cnn = round(recall_score(y_test, y_pred), 4)
precision_cnn = round(precision_score(y_test, y_pred), 4)
f1_cnn = round(f1_score(y_test, y_pred), 4)

# Calculate area under the ROC curve
roc_auc_cnn = round(roc_auc_score(y_test, y_pred), 3)


# Print metrics
print(f'Accuracy: {accuracy_cnn}')
print(f'Precision: {precision_cnn}')
print(f'Recall: {recall_cnn}')
print(f'F1 Score: {f1_cnn}')
print(f'ROC AUC: {roc_auc_cnn}')

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_cnn.history['accuracy'], label='Train Accuracy')
plt.plot(history_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy of CNN Model')
plt.legend()
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_cnn.history['loss'], label='Train loss')
plt.plot(history_cnn.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.title('Training and Validation loss of CNN Model')
plt.legend()
plt.show()

In [ ]:
# Define the LSTM model architecture
model_lstm = Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)),
    Dropout(0.2),  # Add dropout for regularization
    LSTM(units=64, return_sequences=False),
    Dense(64, activation='relu'),  # Add a dense layer with ReLU activation
    Dropout(0.2),  # Add dropout for regularization
    Dense(1, activation='sigmoid')    # For binary classification
])

# Compile the LSTM model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_lstm.summary()

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)


# Train the LSTM model and record training history
history_lstm = model_lstm.fit(X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1),
                              y_train, epochs=300, batch_size=32, validation_split=0.2,
                              verbose=0, callbacks=[early_stopping], shuffle=True)



# Evaluate the model on the test set
y_pred = (model_lstm.predict(X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)) > 0.5).astype("int32")

# Calculate metrics
accuracy_lstm = round(accuracy_score(y_test, y_pred), 4)
precision_lstm = round(precision_score(y_test, y_pred), 4)
recall_lstm = round(recall_score(y_test, y_pred), 4)
f1_lstm = round(f1_score(y_test, y_pred), 4)
roc_auc_lstm = round(roc_auc_score(y_test, y_pred), 4)

# Print metrics
print(f'Accuracy: {accuracy_lstm}')
print(f'Precision: {precision_lstm}')
print(f'Recall: {recall_lstm}')
print(f'F1 Score: {f1_lstm}')
print(f'ROC AUC: {roc_auc_lstm}')

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_lstm.history['accuracy'], label='Train Accuracy')
plt.plot(history_lstm.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy of LSTM Model')
plt.legend()
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_lstm.history['loss'], label='Train loss')
plt.plot(history_lstm.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.title('Training and Validation loss of LSTM Model')
plt.legend()
plt.show()

In [ ]:
# Define the GRU model architecture

model_gru = Sequential([
    GRU(units=128, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)),
    Dropout(0.2),  # Add dropout for regularization
    GRU(units=64, return_sequences=False),
    Dense(64, activation='relu'),  # Add a dense layer with ReLU activation
    Dropout(0.2),  # Add dropout for regularization
    Dense(1, activation='sigmoid')   # For binary classification
])

# Compile the GRU model
model_gru.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'], )

model_gru.summary()

In [ ]:
# Define early stopping callback

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_delta=1E-7)

# Train the GRU model and record training history
history_gru = model_gru.fit(X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1), y_train,
                            epochs=300, batch_size=32, validation_split=0.2, verbose=0,
                            callbacks=[early_stopping])

# Evaluate the model on the test set
y_pred = (model_gru.predict(X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)) > 0.5).astype("int32")

# Calculate metrics
accuracy_gru = round(accuracy_score(y_test, y_pred), 4)
precision_gru = round(precision_score(y_test, y_pred), 4)
recall_gru = round(recall_score(y_test, y_pred), 4)
f1_gru = round(f1_score(y_test, y_pred), 4)
roc_auc_gru = round(roc_auc_score(y_test, y_pred), 4)

# Print metrics
print(f'Accuracy: {accuracy_gru}')
print(f'Precision: {precision_gru}')
print(f'Recall: {recall_gru}')
print(f'F1 Score: {f1_gru}')
print(f'ROC AUC: {roc_auc_gru}')

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_gru.history['accuracy'], label='Train Accuracy')
plt.plot(history_gru.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy of GRU Model')
plt.legend()
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history_gru.history['loss'], label='Train loss')
plt.plot(history_gru.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.title('Training and Validation loss of GRU Model')
plt.legend()
plt.show()

In [ ]:
# Create a dictionary with model names and their corresponding metrics
metrics = {
    'CNN': {'Accuracy': accuracy_cnn, 'Precision': precision_cnn, 'Recall': recall_cnn, 'F1 Score': f1_cnn, 'ROC': roc_auc_cnn},
    'LSTM': {'Accuracy': accuracy_lstm, 'Precision': precision_lstm, 'Recall': recall_lstm, 'F1 Score': f1_lstm, 'ROC': roc_auc_lstm},
    'GRU': {'Accuracy': accuracy_gru, 'Precision': precision_gru, 'Recall': recall_gru, 'F1 Score': f1_gru, 'ROC': roc_auc_gru},
}

# Convert the dictionary to a pandas DataFrame
metrics_table = pd.DataFrame(metrics).T

# Display the accuracy table
print(metrics_table)

In [ ]:
stop_run = time.time()

total_time = stop_run - start_run

project_time = (total_time/60)

print("totaltime of iteration is: %0.1f minutes" % (project_time))